# NTK classifier for Cora

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from exp_ntk_certify_single import run

In [3]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [4]:
seed = 0

data_params = dict(
    dataset = "csbm",
    learning_setting = "inductive", # or "transdructive"
    specification = dict(
        classes = 2,
        n_trn_labeled = 600,
        n_trn_unlabeled = 0,
        n_val = 200,
        n_test = 200,
        sigma = 1,
        avg_within_class_degree = 1.58 * 2,
        avg_between_class_degree = 0.37 * 2,
        K = 1.5,
        seed = 0 # used to generate the dataset & data split
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
)

certificate_params = dict(
    perturbation_model = "l0",
    delta = 5 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [5]:
def run_exp(n_seeds, data_params, model_params, certificate_params,
            verbosity_params, other_params):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    for seed in range(n_seeds):
        data_params["specification"]["seed"] = seed
        res = run(data_params, model_params, certificate_params,
                  verbosity_params, other_params, seed)
        acc_l.append(res["accuracy"])
        min_ypred.append(res["min_ypred"])
        max_ypred.append(res["max_ypred"])
        min_ntklabeled.append(res["min_ntklabeled"])
        max_ntklabeled.append(res["max_ntklabeled"])
        min_ntkunlabeled.append(res["min_ntkunlabeled"])
        max_ntkunlabeled.append(res["max_ntkunlabeled"])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [7]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "csbm"
other_params["device"] = 0
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
certificate_params["delta"] = 0.01
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-01-18 09:55:11 (INFO): Starting experiment exp_ntk_certify_single with configuration:
2024-01-18 09:55:11 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'inductive', 'specification': {'classes': 2, 'n_trn_labeled': 600, 'n_trn_unlabeled': 0, 'n_val': 200, 'n_test': 200, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 0}}
2024-01-18 09:55:11 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'cache_size': 10000}
2024-01-18 09:55:11 (INFO): certification_params: {'perturbation_model': 'linf', 'delta': 0.01, 'method': 'XXT'}
2024-01-18 09:55:11 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-01-18 09:55:11 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-01-18 09:55:11 (INFO): seed: 0
2024-01-18 09:55:11 (INFO): Currently on gpu device cuda:0
2024-01-18 09:55:11 (INFO): CSBM(p=0.0063263

CSBM mu:
[0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342]


2024-01-18 09:55:11 (INFO): X.min(): -3.859495162963867
2024-01-18 09:55:11 (INFO): X.max(): 4.109212875366211
2024-01-18 09:55:11 (INFO): X_rowsum.mean(): 0.33397936742261686
2024-01-18 09:55:11 (INFO): X_rowsum.median(): 0.7591041489504278
2024-01-18 09:55:11 (INFO): X_rowsum.min(): -16.377630546689034
2024-01-18 09:55:11 (INFO): X_rowsum.max(): 17.149057352915406
2024-01-18 09:55:11 (INFO): Accuracy 0.8399999737739563


Sig.mean(): 0.030798085060794136
Sig.min(): -14.949451596673214
Sig.max(): 32.95754895220645
Depth 0
E_der.min(): 0.18289396597227744
E_der.max(): 0.9998576474901678
Sig.mean(): 1.8967366306355231
Sig.min(): 0.3569972147566615
Sig.max(): 32.95617481733851
Sig.mean(): 0.025097406718435484
Sig.min(): -14.462656274780015
Sig.max(): 31.56303964237527
Depth 0
E_der.min(): 0.18234846643512442
E_der.max(): 0.9998576474901678
Sig.mean(): 1.5459834786357913
Sig.min(): 0.44621182065889875
Sig.max(): 31.561723650297893
Sig.mean(): 0.025097406718435484
Sig.min(): -14.462656274780015
Sig.max(): 31.56303964237527
Sig[:,idx_adv]: -0.08367021490030059
Sig_ub.mean(): 0.025354577189939487
Sig_ub.min(): -14.462656274780015
Sig_ub.max(): 31.56303964237527
Sig_ub[:,idx_adv]: -0.04961734612992686
Sig_lb.mean(): 0.02484023624693149
Sig_lb.min(): -14.462656274780015
Sig_lb.max(): 31.56303964237527
Sig_lb[:,idx_adv]: -0.11772308367067431
tensor(1.2113, device='cuda:0', dtype=torch.float64)
tensor(1.2113, devic

KeyboardInterrupt: 

In [10]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "csbm"
other_params["device"] = 0
certificate_params["method"] = "XXT"
certificate_params["perturbation_model"] = "linf"
certificate_params["delta"] = 0.01
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-01-18 09:58:16 (INFO): Starting experiment exp_ntk_certify_single with configuration:
2024-01-18 09:58:16 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'inductive', 'specification': {'classes': 2, 'n_trn_labeled': 600, 'n_trn_unlabeled': 0, 'n_val': 200, 'n_test': 200, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 0}}
2024-01-18 09:58:16 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'cache_size': 10000}
2024-01-18 09:58:16 (INFO): certification_params: {'perturbation_model': 'linf', 'delta': 0.01, 'method': 'XXT'}
2024-01-18 09:58:16 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-01-18 09:58:16 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-01-18 09:58:16 (INFO): seed: 0
2024-01-18 09:58:16 (INFO): Currently on gpu device cuda:0
2024-01-18 09:58:16 (INFO): CSBM(p=0.0063263

CSBM mu:
[0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342 0.16366342
 0.16366342 0.16366342 0.16366342]


2024-01-18 09:58:16 (INFO): Accuracy 0.8399999737739563


Sig.mean(): 0.030798085060794136
Sig.min(): -14.949451596673214
Sig.max(): 32.95754895220645
Depth 0
E_der.min(): 0.18289396597227744
E_der.max(): 0.9998576474901678
Sig.mean(): 1.8967366306355231
Sig.min(): 0.3569972147566615
Sig.max(): 32.95617481733851
Sig.mean(): 0.025097406718435484
Sig.min(): -14.462656274780015
Sig.max(): 31.56303964237527
Depth 0
E_der.min(): 0.18234846643512442
E_der.max(): 0.9998576474901678
Sig.mean(): 1.5459834786357913
Sig.min(): 0.44621182065889875
Sig.max(): 31.561723650297893
Sig.mean(): 0.025097406718435484
Sig.min(): -14.462656274780015
Sig.max(): 31.56303964237527
Sig[:,idx_adv]: -0.08367021490030059
Sig_ub.mean(): 0.025354577189939487
Sig_ub.min(): -14.462656274780015
Sig_ub.max(): 31.56303964237527
Sig_ub[:,idx_adv]: -0.04961734612992686
Sig_lb.mean(): 0.02484023624693149
Sig_lb.min(): -14.462656274780015
Sig_lb.max(): 31.56303964237527
Sig_lb[:,idx_adv]: -0.11772308367067431
tensor(1.2113, device='cuda:0', dtype=torch.float64)
tensor(1.2113, devic

KeyboardInterrupt: 